### Linear Models in TensorFlow: Timing Factors and Smart-Beta Strategies

**Project Goal**: To build a neural network, under a linear model, that aims to predict the future returns of the momentum factor.

**Data SOurce**

- We are gping to take as inputs the returns from a momentum factor. Using these inputs, we will then aim to predict next period momentum factor returns using as inputs past returns. 

- Data source is Prof. Ken French's Data Library: [Daily returns of 10 Portfolios Formed Daily on Momentum.](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_10_port_form_pr_12_2_daily.html)

In [1]:
import numpy as np
import pandas as pd

**Timing Momentum with (Linear Regression) Neural Networks**

In [2]:
route = "10_Portfolios_Prior_12_2_Daily.CSV"

In [ ]:
# Read csv with skipped rows
df = pd.read_csv(route, index_col=0)
# Format the date index
df.index = pd.to_datetime(df.index, format="%Y%m%d")
# Build the MOM stratgy: long "Hi PRIOR" and SHORT "Lo PRIOR"
df["MOM"] = df["Hi PRIOR"] - df["Lo PRIOR"]
df.head()

ValueError: time data "  Average Equal Weighted Returns -- Daily" doesn't match format "%Y%m%d", at position 25781. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.